In [ ]:
#@title Imports { form-width: "20%" }

import numpy as np
import urllib.request
import re
from collections import Counter
import itertools
import hashlib


def get_input(day):
  return urllib.request.urlopen(
      f'https://raw.githubusercontent.com/SantoSimone/Advent-of-Code/master/'
      f'2016/input_files/input{day}.txt'
      ).read().decode('utf-8')

def get_input_as_lines(day):
  return get_input(day).split('\n')[:-1]

def parse_ints(text):
  return [int(x) for x in re.findall(r'\d+', text)]


# Day 1


In [ ]:
#@title Part 1 { form-width: "20%" }
def no_time_for_taxicab(input):
  # Parsing inputs
  moves = [(el[0], el[1:]) for el in [split.replace(' ', '') 
            for split in input.split(sep=',')]] # (dir, num)

  # Starting pos is center of plane heading North
  pos = 0
  head = 1j
  visited = [pos]

  for direction, steps in moves:    
    # turn right is a 90° clockwise, while left is 90° anti-clockwise
    head *= -1j if direction == 'R' else 1j
    for i in range(int(steps)):
      pos += 1 * head
      visited.append(pos)

  return abs(pos.real) + abs(pos.imag)


# It's day 1
day = 1

input = get_input(day)

no_time_for_taxicab(input)


In [ ]:
#@title Part 2 { form-width: "20%" }
# We just need to add a check in the function

def no_time_for_taxicab_pt2(input):
  # Parsing inputs
  moves = [(el[0], el[1:]) for el in [split.replace(' ', '') 
            for split in input.split(sep=',')]] # (dir, num)

  # Starting pos is center of plane heading North
  pos = 0
  head = 1j
  visited = [pos]

  for direction, steps in moves:    
    # turn right is a 90° clockwise, while left is 90° anti-clockwise
    head *= -1j if direction == 'R' else 1j
    for i in range(int(steps)):
      pos += 1 * head
      if pos in visited:
        return abs(pos.real) + abs(pos.imag)
      visited.append(pos)

  return abs(pos.real) + abs(pos.imag)


# It's day 1
no_time_for_taxicab_pt2(input)

# Day 2


In [ ]:
#@title Part 1 { form-width: "20%" }
# It's day 2
day = 2

# Split file into lines
lines = get_input_as_lines(day)

keypad = np.array([
  ['1', '2', '3'],
  ['4', '5', '6'],
  ['7', '8', '9']
])

# Encoding movements (Row, Col)
moves = {
  'U' : np.array([-1, 0]),
  'D' : np.array([+1, 0]),
  'L' : np.array([0, -1]),
  'R' : np.array([0, +1])
}
pos = np.array([1, 1])
code = ''

for line in lines:
  for move in line:
    pos += moves[move]
    pos = np.clip(pos, 0, keypad.shape[0] -1)    
  
  # End of line -> store actual value
  code += keypad[pos[0], pos[1]]

# Final code to bathroom
print(code)

In [ ]:
#@title Part 2 { form-width: "20%" }
# New keypad
keypad = np.array([
  ['0', '0', '1', '0', '0'],
  ['0', '2', '3', '4', '0'],
  ['5', '6', '7', '8', '9'],
  ['0', 'A', 'B', 'C', '0'],
  ['0', '0', 'D', '0', '0'],
])

# New rules
def should_i_stay_or_should_i_go(pos, move):
  move = pos + move
  move = np.clip(move, 0, 4)
  
  return pos if keypad[move[0], move[1]] == '0' else move

pos = np.array([2, 0])
code = ''

for line in lines:
  for move in line:
    pos = should_i_stay_or_should_i_go(pos, moves[move])
  
  code += keypad[pos[0], pos[1]]

  
print(code)


# Day 3

In [ ]:
#@title Part 1 { form-width: "20%" }

def check_triangle(tr):
  # Returns True if valid False otherwise
  x, y, z = sorted(tr)
  return z < x + y


# It's day 3
day = 3

# Split file into lines
lines = get_input_as_lines(day)
# Each line is a triangle
step = [line.split(' ') for line in lines]
# Filtering out empty splits
triangles = [parse_ints(l) for l in lines]

sum(map(check_triangle, triangles))

In [ ]:
#@title Part 2 { form-width: "20%" }

new_triangles = [
                 tr 
                 for i in range(0, len(triangles), 3)
                 for tr in zip(triangles[i], triangles[i+1], triangles[i+2])
                 ]

sum(map(check_triangle, new_triangles))

# Day 4

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_room(room):
  # Parse chars (with dashes), sector id and checksum for each room entry
  return re.match(r'([a-z-]+)-(\d+)\[([a-z]+)\]', room).groups()

def check_room(room):
  chars, id, checksum = parse_room(room)
  # collections.Counter obj performs exactly what we want
  counts = Counter(chars.replace('-', ''))
  # Getting real checksum
  real = ''.join(
      [v[0] for v in sorted(counts.items(), key=lambda x: (-x[1], x[0]))][:5]
      )
  return int(id) if real == checksum else 0


# It's day 4
day = 4

# Split file into lines
lines = get_input_as_lines(day)
# Each line is a room
rooms = [line for line in lines]

sum = np.sum([check_room(r) for r in rooms])
sum

In [ ]:
#@title Part 2 { form-width: "20%" }

def decrypt(room):
  chars, id, checksum = parse_room(room)
  # First, we decrypt a single word, then we concatenates with whitespaces
  ret = ' '.join([ 
                ''.join([chr(97 + (ord(ch) - 97 + int(id)) % 26)
                for ch in word])
        for word in chars.split('-') 
        ])
  return ret, int(id)

for room in rooms:
  msg, id = decrypt(room)
  if 'north' in msg:
    print(id)


# Day 5

In [ ]:
#@title Part 1 { form-width: "20%" }
# It's day 5
day = 5

# my input
input = 'ojvtpuvg'

# PART ONE
i = 0
pw = ''
while True:
  ex = f'{input}{i}'
  code = hashlib.md5(ex.encode()).hexdigest()
  if code.startswith('00000'):
    pw += code[5]
    if len(pw) == 8: break

  i += 1

print(f'Final password: {pw}')

In [ ]:
#@title Part 2 { form-width: "20%" }
# It takes longer, so we do some random stuff to make it fancier
input = 'ojvtpuvg'
i = 0
pw = ['_'] * 8
changed = 0
while True:
  ex = f'{input}{i}'
  code = hashlib.md5(ex.encode()).hexdigest()
  if code.startswith('00000') and code[5].isdigit() and int(code[5]) < 8 and pw[int(code[5])] == '_':    
    changed += 1
    pw[int(code[5])] = code[6]
    print(f'\rDecrypted password: {"".join(pw)}', end='')
    if changed == 8: break
  i += 1

# Day 6


In [ ]:
#@title Part 1 { form-width: "20%" }
# It's day 6
day = 6

lines = get_input_as_lines(day)

positions = zip(*lines)
counters = [Counter(pos) for pos in positions]

# getting top 1 occurence
msg = ''.join([count.most_common(1)[0][0] for count in counters])

print(msg)

In [ ]:
#@title Part 2 { form-width: "20%" }
# Difference in p2 is that we take the least common

# getting all occurences sorted, then getting the last
msg = ''.join([count.most_common()[-1][0] for count in counters])
print(msg)

# Day 7

In [ ]:
#@title Part 1 { form-width: "20%" }

def check_abba(word):
    splits = [word[i:i + 4] for i in range(len(word) - 3)]
    return any([s[:2] == ''.join(reversed(s[2:])) and s[0] != s[1] 
                for s in splits])


def support_TLS(line):
    splits = re.split(r'\[|\]', line)
    hypernets = splits[1::2]
    supernets = splits[0::2]

    return not any(map(check_abba, hypernets)) \
           and any(map(check_abba, supernets))

# It's day 7
day = 7
lines = get_input_as_lines(day)
s = np.sum([support_TLS(line) for line in lines])
print(f'TLS compliant IPs: {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def check_aba(word, hypernets):
    splits = [word[i:i + 3] for i in range(len(word) - 2)]
    return any([s[0] == s[2] != s[1] and any([check_bab(''.join([s[1], s[0], s[1]]), br) for br in hypernets])
                for s in splits])

def check_bab(check, word):
    return check in word

def support_SSL(line):
    splits = re.split(r'\[|\]', line)
    hypernets = splits[1::2]
    supernets = splits[0::2]

    return any([check_aba(s, hypernets) for s in supernets])


lines = get_input_as_lines(day)
s = np.sum([support_SSL(line) for line in lines])
print(f'SSL compliant IPs: {s}')

# Day 8

In [ ]:
#@title Part 1 { form-width: "20%" }

def exec(line, screen):
  if re.findall('rect', line):
      w, h = parse_ints(line)
      screen[:h, :w] = 1
  elif re.findall('rotate column', line):
      col, shift = parse_ints(line)
      screen[:, col] = np.roll(screen[:, col], shift, axis=0)
  elif re.findall('rotate row', line):
      row, shift = parse_ints(line)
      screen[row, :] = np.roll(screen[row, :], shift, axis=0)

  return screen

# It's day 8
day = 8
lines = get_input_as_lines(day)
screen = np.zeros((6, 50), dtype=np.int32)
for line in lines:
    screen = exec(line, screen)
print(f'Final number of lights: {np.sum(screen, (0,1))}')

In [ ]:
#@title Part 2 { form-width: "20%" }
def print_screen(screen):
  for i in range(screen.shape[0]):
    for j in range(screen.shape[1]):
      if screen[i, j]:
        print('#', end='')
      else:
        print(' ', end='')
    print(' ')

print('Final screen: ')
print_screen(screen)

# Day 9

In [ ]:
#@title Part 1 { form-width: "20%" }

def next_marker(txt):
  search = re.search(r'\((\d+)x(\d+)\)|$', txt)
  start, stop = search.span()
  if start == stop: return start, stop, 0, 0
  length, reps = map(int, search.groups())
  return start, stop, length, reps


def calc_len(txt):
  total = 0
  while True:
      start, stop, length, reps = next_marker(txt)
      total += start + length * reps
      i = stop + length
      if i >= len(txt): break
      txt = txt[i:]
  return total

# It's day 9
day = 9
txt = ''.join(get_input_as_lines(day))
length = calc_len(txt)
print(f'Total length: {length}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# We need to add a bit of recursion here
def calc_len_v2(txt):
  total = 0
  while True:
      start, stop, length, reps = next_marker(txt)
      if start == stop: return start
      total += start + calc_len_v2(txt[stop:length + stop]) * reps
      i = stop + length
      if i >= len(txt): break
      txt = txt[i:]
  return total

length = calc_len_v2(txt)
print(f'Total length: {length}')